# Running hyperparameter optimization on Chemprop model using RayTune

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/chemprop/chemprop/blob/main/examples/hpopting.ipynb)

In [1]:
# Install chemprop from GitHub if running in Google Colab
import os

if os.getenv("COLAB_RELEASE_TAG"):
    try:
        import chemprop
    except ImportError:
        !git clone https://github.com/chemprop/chemprop.git
        %cd chemprop
        !pip install ".[hpopt]"
        %cd examples

Cloning into 'chemprop'...
remote: Enumerating objects: 24987, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 24987 (delta 115), reused 44 (delta 44), pack-reused 24800 (from 3)
Receiving objects: 100% (24987/24987), 817.53 MiB | 25.45 MiB/s, done.
Resolving deltas: 100% (17934/17934), done.
/content/chemprop
Processing /content/chemprop
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/66

## Import packages

In [2]:
from pathlib import Path

import pandas as pd
from lightning import pytorch as pl
import ray
from ray import tune
from ray.train import CheckpointConfig, RunConfig, ScalingConfig
from ray.train.lightning import (RayDDPStrategy, RayLightningEnvironment,
                                 RayTrainReportCallback, prepare_trainer)
from ray.train.torch import TorchTrainer
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import FIFOScheduler

from chemprop import data, featurizers, models, nn

In [3]:
chemprop_dir = Path.cwd().parent
input_path = "/content/Augmented_data.csv" # path to your data .csv file
num_workers = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'SMILES ' # name of the column containing SMILES strings
target_columns = ['Status'] # list of names of the columns containing targets

hpopt_save_dir = Path.cwd() / "hpopt" # directory to save hyperopt results
hpopt_save_dir.mkdir(exist_ok=True)

## Load data

In [4]:
df_input = pd.read_csv(input_path)
df_input

,Unnamed: 0,Catalog Number,SMILES,Active/Inactive,Remark,Status
0,0,S7718,CN(C)CCNC(=O)C1=CC=CN2C(=O)C3=C(C=C4C=CC=CC4=C...,Active,Training,1
1,1,S2679,Cl.CN1CCC(C(O)C1)C2=C3OC(=CC(=O)C3=C(O)C=C2O)C...,Active,Training,1
2,2,S7511,CC(C)C1=C2C=C(C=CC2=N[N]1C)C3=CC=NC(=N3)NC4CCC...,Active,Training,1
3,3,S1393,COC1=CC=CC2=C1C(=O)C3=C(O)C4=C(CC(O)(CC4OC5CC(...,Active,Training,1
4,4,S7648,Cl.Cl.CC(CN(C)C)C1=CC=C(C=C1)C2=C3C(=C(C)C=C2O...,Active,Training,1
...,...,...,...,...,...,...
4681,4724,NaN,[1*]C(=O)C(C)CC.[13*]C1CC(O)CC(=O)O1.[3*]O[3*]...,Active,NaN,1
4682,4725,NaN,[15*]C1CCC=C2C=CC(C)C([15*])C21.[1*]C(=O)C(C)C...,Active,NaN,1
4683,4726,NaN,[9*]n1c(=O)c([16*])cc2c(C)nc(N)nc21.[3*]O[3*]....,Active,NaN,1
4684,4727,NaN,[14*]c1ccc([16*])cn1.[4*]CCO.[15*]C1CCC([15*])...,Active,NaN,1


In [5]:
smis = df_input.loc[:, smiles_column].values
ys = df_input.loc[:, target_columns].values

## Make data points, splits, and datasets

In [6]:
all_data = []
for smi, y in zip(smis, ys):
    try:
        datapoint = data.MoleculeDatapoint.from_smi(smi, y)
        all_data.append(datapoint)
    except RuntimeError:
        print(f"Skipping invalid SMILES: {smi}")

[10:57:44] SMILES Parse Error: syntax error while parsing: COC(=O)NC(C)CNC1=NC=CC(=N1)C2=C[N](N=C2C3=C(F)C(=CC(=C3)Cl)N[S;v6](C)(=O)=O)C(C)C
[10:57:44] SMILES Parse Error: check for mistakes around position 63:
[10:57:44] C(F)C(=CC(=C3)Cl)N[S;v6](C)(=O)=O)C(C)C
[10:57:44] ~~~~~~~~~~~~~~~~~~~~^
[10:57:44] SMILES Parse Error: Failed parsing SMILES 'COC(=O)NC(C)CNC1=NC=CC(=N1)C2=C[N](N=C2C3=C(F)C(=CC(=C3)Cl)N[S;v6](C)(=O)=O)C(C)C' for input: 'COC(=O)NC(C)CNC1=NC=CC(=N1)C2=C[N](N=C2C3=C(F)C(=CC(=C3)Cl)N[S;v6](C)(=O)=O)C(C)C'
[10:57:44] SMILES Parse Error: syntax error while parsing: O.O.O.OC1=CC=CN|2=C1C(=O)O[VH]3|2(O)(=O)OC(=O)C4=NC=CC=C4O3
[10:57:44] SMILES Parse Error: check for mistakes around position 16:
[10:57:44] O.O.O.OC1=CC=CN|2=C1C(=O)O[VH]3|2(O)(=O)O
[10:57:44] ~~~~~~~~~~~~~~~^
[10:57:44] SMILES Parse Error: Failed parsing SMILES 'O.O.O.OC1=CC=CN|2=C1C(=O)O[VH]3|2(O)(=O)OC(=O)C4=NC=CC=C4O3' for input: 'O.O.O.OC1=CC=CN|2=C1C(=O)O[VH]3|2(O)(=O)OC(=O)C4=NC=CC=C4O3'
[10:57:44] SMIL

Skipping invalid SMILES: COC(=O)NC(C)CNC1=NC=CC(=N1)C2=C[N](N=C2C3=C(F)C(=CC(=C3)Cl)N[S;v6](C)(=O)=O)C(C)C
Skipping invalid SMILES: O.O.O.OC1=CC=CN|2=C1C(=O)O[VH]3|2(O)(=O)OC(=O)C4=NC=CC=C4O3
Skipping invalid SMILES: Cl.CC1=CC(=CC(=C1)C[S;v6](=O)(=O)C2=CC=CC=C2)OCC3=CC=C(CN4CCCC4CO)C=C3
Skipping invalid SMILES: N[S;v6](=O)(=O)OCC1CC(CC1O)[N]2C=CC3=C(NC4CCC5=CC=CC=C45)N=CN=C23


[10:57:44] SMILES Parse Error: extra open parentheses while parsing: CCC(C)C1NC(=O)C(CC2=CC=C(O)C=C2)NC(=O)C(CC(C)C)NC(=O)C3CCCN3C(=O)C(CSSCC(NC(=O)C(NC(=O)C4CCCN4C(=O)C(CC(O)=O)NC(=O)C(CC5=CC=C(O)C=C5)NC(=O)C(CO)NC1=O)C(C)C)C(=O)NC(CCCNC(N)=N)C(=O)NC(CCCNC(N)=N)C(=O)N
[10:57:44] SMILES Parse Error: check for mistakes around position 67:
[10:57:44] )NC(=O)C3CCCN3C(=O)C(CSSCC(NC(=O)C(NC(=O)
[10:57:44] ~~~~~~~~~~~~~~~~~~~~^
[10:57:44] SMILES Parse Error: Failed parsing SMILES 'CCC(C)C1NC(=O)C(CC2=CC=C(O)C=C2)NC(=O)C(CC(C)C)NC(=O)C3CCCN3C(=O)C(CSSCC(NC(=O)C(NC(=O)C4CCCN4C(=O)C(CC(O)=O)NC(=O)C(CC5=CC=C(O)C=C5)NC(=O)C(CO)NC1=O)C(C)C)C(=O)NC(CCCNC(N)=N)C(=O)NC(CCCNC(N)=N)C(=O)N' for input: 'CCC(C)C1NC(=O)C(CC2=CC=C(O)C=C2)NC(=O)C(CC(C)C)NC(=O)C3CCCN3C(=O)C(CSSCC(NC(=O)C(NC(=O)C4CCCN4C(=O)C(CC(O)=O)NC(=O)C(CC5=CC=C(O)C=C5)NC(=O)C(CO)NC1=O)C(C)C)C(=O)NC(CCCNC(N)=N)C(=O)NC(CCCNC(N)=N)C(=O)N'
[10:57:44] SMILES Parse Error: syntax error while parsing: CC(C)(C)C[N]1C(=NC2=CC=C(N=C12)C3=C(N=C([NH]3

Skipping invalid SMILES: CCC(C)C1NC(=O)C(CC2=CC=C(O)C=C2)NC(=O)C(CC(C)C)NC(=O)C3CCCN3C(=O)C(CSSCC(NC(=O)C(NC(=O)C4CCCN4C(=O)C(CC(O)=O)NC(=O)C(CC5=CC=C(O)C=C5)NC(=O)C(CO)NC1=O)C(C)C)C(=O)NC(CCCNC(N)=N)C(=O)NC(CCCNC(N)=N)C(=O)N
Skipping invalid SMILES: CC(C)(C)C[N]1C(=NC2=CC=C(N=C12)C3=C(N=C([NH]3)C(C)(C)C)C4=CC=C(F)C=C4)N.C[S;v6](O)(=O)=O


In [7]:
mols = [d.mol for d in all_data]  # RDkit Mol objects are use for structure based splits
train_indices, val_indices, test_indices = data.make_split_indices(mols, "random", (0.8, 0.1, 0.1))
train_data, val_data, test_data = data.split_data_by_indices(
    all_data, train_indices, val_indices, test_indices
)

In [8]:
from chemprop.featurizers import SimpleMoleculeMolGraphFeaturizer
from rdkit.Chem import Descriptors, rdMolDescriptors
class CustomFeaturizer(SimpleMoleculeMolGraphFeaturizer):
    def featurize(self, mol):
        atom_features = []
        for atom in mol.GetAtoms():
            atom_features.append([
                atom.GetAtomicNum(),
                atom.GetNumBonds(),
                atom.GetFormalCharge(),
                atom.GetChiralTag(),
                atom.GetTotalNumHs(),
                atom.GetHybridization(),
                atom.GetIsAromatic(),
                atom.GetMass()
            ])

        bond_features = []
        for bond in mol.GetBonds():
            bond_features.append([
                bond.GetBondType(),
                bond.GetIsConjugated(),
                bond.IsInRing(),
                bond.GetStereo()
            ])

        mol_features = []
        for descriptor_name, descriptor_func in Descriptors.descList:
            mol_features.append(descriptor_func(mol))

        additional_features = []
        for descriptor_name, descriptor_func in rdMolDescriptors.descList:
            additional_features.append(descriptor_func(mol))

        return atom_features + bond_features + mol_features + additional_features

featurizer = CustomFeaturizer()

# === Create datasets and loaders ===
train_dset = data.MoleculeDataset(train_data[0], featurizer)
scaler = train_dset.normalize_targets()
val_dset = data.MoleculeDataset(val_data[0], featurizer)
val_dset.normalize_targets(scaler)
test_dset = data.MoleculeDataset(test_data[0], featurizer)

# Define helper function to train the model

In [9]:
def train_model(config, train_dset, val_dset, num_workers, scaler):

    # config is a dictionary containing hyperparameters used for the trial
    depth = int(config["depth"])
    ffn_hidden_dim = int(config["ffn_hidden_dim"])
    ffn_num_layers = int(config["ffn_num_layers"])
    message_hidden_dim = int(config["message_hidden_dim"])

    train_loader = data.build_dataloader(train_dset, num_workers=num_workers, shuffle=True)
    val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)

    mp = nn.BondMessagePassing(d_h=message_hidden_dim, depth=depth)
    agg = nn.MeanAggregation()
    output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
    ffn = nn.BinaryClassificationFFN(output_transform=output_transform, input_dim=message_hidden_dim, hidden_dim=ffn_hidden_dim, n_layers=ffn_num_layers)
    batch_norm = True
    metric_list = [nn.metrics.BinaryF1Score()]
    model = models.MPNN(mp, agg, ffn, batch_norm, metric_list)

    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        max_epochs=20, # number of epochs to train for
        # below are needed for Ray and Lightning integration
        strategy=RayDDPStrategy(),
        callbacks=[RayTrainReportCallback()],
        plugins=[RayLightningEnvironment()],
    )

    trainer = prepare_trainer(trainer)
    trainer.fit(model, train_loader, val_loader)


## Define parameter search space

In [10]:

search_space = {
    "depth": tune.qrandint(lower=2, upper=6, q=1),
    "ffn_hidden_dim": tune.qrandint(lower=300, upper=2400, q=100),
    "ffn_num_layers": tune.qrandint(lower=1, upper=3, q=1),
    "message_hidden_dim": tune.qrandint(lower=300, upper=2400, q=100),
}

In [ ]:


ray.shutdown()
ray.init()

# define the scaler before using it

scheduler = FIFOScheduler()

# Scaling config controls the resources used by Ray
scaling_config = ScalingConfig(
    num_workers=1,
    use_gpu= False, # change to True if you want to use GPU
)

# Checkpoint config controls the checkpointing behavior of Ray
checkpoint_config = CheckpointConfig(
    num_to_keep=1, # number of checkpoints to keep
    checkpoint_score_attribute="val_loss", # Save the checkpoint based on this metric
    checkpoint_score_order="min", # Save the checkpoint with the lowest metric value
)

run_config = RunConfig(
    checkpoint_config=checkpoint_config,
    storage_path=hpopt_save_dir / "ray_results", # directory to save the results
)

ray_trainer = TorchTrainer(
    lambda config: train_model(
        config, train_dset, val_dset, num_workers, scaler
    ),
    scaling_config=scaling_config,
    run_config=run_config,
)

search_alg = HyperOptSearch(
    n_initial_points=1, # number of random evaluations before tree parzen estimators
    random_state_seed=42,
)

# OptunaSearch is another search algorithm that can be used
# search_alg = OptunaSearch()

tune_config = tune.TuneConfig(
    metric="val_loss",
    mode="min",
    num_samples=20, # number of trials to run
    scheduler=scheduler,
    search_alg=search_alg,
    trial_dirname_creator=lambda trial: str(trial.trial_id), # shorten filepaths

)

tuner = tune.Tuner(
    ray_trainer,
    param_space={
        "train_loop_config": search_space,
    },
    tune_config=tune_config,
)

# Start the hyperparameter search
results = tuner.fit()


2025-05-05 10:58:32,011	INFO worker.py:1888 -- Started a local Ray instance.


+---------------------------------------------------------------------+
| Configuration for experiment     TorchTrainer_2025-05-05_10-58-35   |
+---------------------------------------------------------------------+
| Search algorithm                 SearchGenerator                    |
| Scheduler                        FIFOScheduler                      |
| Number of trials                 20                                 |
+---------------------------------------------------------------------+

View detailed results here: /content/chemprop/examples/hpopt/ray_results/TorchTrainer_2025-05-05_10-58-35
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-05-05_10-58-28_699870_493/artifacts/2025-05-05_10-58-35/TorchTrainer_2025-05-05_10-58-35/driver_artifacts`

Trial status: 1 PENDING
Current time: 2025-05-05 10:58:35. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+--------------------------------------------------------------

(TorchTrainer pid=2818) Started distributed worker processes: 
(TorchTrainer pid=2818) - (node_id=85d1826776a8b04eb71395ec08130d8b282f617a724b5c6551ccdd9f, ip=172.28.0.12, pid=2908) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=2908) Setting up process group for: env:// [rank=0, world_size=1]



Trial TorchTrainer_c410a352 started with configuration:
+---------------------------------------------+
| Trial TorchTrainer_c410a352 config          |
+---------------------------------------------+
| train_loop_config/depth                   2 |
| train_loop_config/ffn_hidden_dim       2200 |
| train_loop_config/ffn_num_layers          2 |
| train_loop_config/message_hidden_dim    400 |
+---------------------------------------------+


(RayTrainWorker pid=2908) Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
(RayTrainWorker pid=2908) GPU available: False, used: False
(RayTrainWorker pid=2908) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=2908) HPU available: False, using: 0 HPUs


...

Epoch 3:  12%|█▏        | 7/59 [00:55<06:49,  0.13it/s, v_num=0, train_loss_step=-6.75e+5, val_loss=-2.29e+5, train_loss_epoch=-1.6e+5]


2025-05-05 11:47:01,815	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2025-05-05 11:47:01,830	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/content/chemprop/examples/hpopt/ray_results/TorchTrainer_2025-05-05_10-58-35' in 0.0137s.


Trial status: 3 TERMINATED | 2 RUNNING | 1 PENDING
Current time: 2025-05-05 11:47:01. Total running time: 48min 26s
Logical resource usage: 2.0/2 CPUs, 0/0 GPUs
Current best trial: 8da927e1 with val_loss=-533901632.0 and params={'train_loop_config': {'depth': 2, 'ffn_hidden_dim': 2000, 'ffn_num_layers': 2, 'message_hidden_dim': 500}}
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name              status         ...loop_config/depth     ...ig/ffn_hidden_dim     ...ig/ffn_num_layers     ...essage_hidden_dim     iter     total time (s)          train_loss     train_loss_step     val/f1            val_loss |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

(RayTrainWorker pid=15066) Loading `train_dataloader` to estimate number of stepping batches.
(RayTrainWorker pid=15066) 
(RayTrainWorker pid=15066)   | Name            | Type                    | Params | Mode 
(RayTrainWorker pid=15066) --------------------------------------------------------------------
(RayTrainWorker pid=15066) 0 | message_passing | BondMessagePassing      | 3.1 M  | train
(RayTrainWorker pid=15066) 1 | agg             | MeanAggregation         | 0      | train
(RayTrainWorker pid=15066) 2 | bn              | BatchNorm1d             | 2.4 K  | train
(RayTrainWorker pid=15066) 3 | predictor       | BinaryClassificationFFN | 1.7 M  | train
(RayTrainWorker pid=15066) 4 | X_d_transform   | Identity                | 0      | train
(RayTrainWorker pid=15066) 5 | metrics         | ModuleList              | 0      | train
(RayTrainWorker pid=15066) --------------------------------------------------------------------
(RayTrainWorker pid=15066) 4.8 M     Trainable params
(R

Sanity Checking: |          | 0/? [00:00<?, ?it/s]
Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:05<00:05,  0.20it/s]


## Hyperparameter optimization results

In [ ]:
results

ResultGrid<[
  Result(
    metrics={'train_loss': 0.09904231131076813, 'train_loss_step': 0.16821686923503876, 'val/rmse': 0.8613682389259338, 'val/mae': 0.7006751298904419, 'val_loss': 0.7419552206993103, 'train_loss_epoch': 0.09904231131076813, 'epoch': 19, 'step': 40},
    path='/home/knathan/chemprop/examples/hpopt/ray_results/TorchTrainer_2024-10-22_09-03-37/f1a6e41a',
    filesystem='local',
    checkpoint=Checkpoint(filesystem=local, path=/home/knathan/chemprop/examples/hpopt/ray_results/TorchTrainer_2024-10-22_09-03-37/f1a6e41a/checkpoint_000019)
  ),
  Result(
    metrics={'train_loss': 0.06969495117664337, 'train_loss_step': 0.11989812552928925, 'val/rmse': 0.902579665184021, 'val/mae': 0.7176367044448853, 'val_loss': 0.8146500587463379, 'train_loss_epoch': 0.06969495117664337, 'epoch': 19, 'step': 40},
    path='/home/knathan/chemprop/examples/hpopt/ray_results/TorchTrainer_2024-10-22_09-03-37/d775c15d',
    filesystem='local',
    checkpoint=Checkpoint(filesystem=local, pat

In [ ]:
# results of all trials
result_df = results.get_dataframe()
result_df

,train_loss,train_loss_step,val/rmse,val/mae,val_loss,train_loss_epoch,epoch,step,timestamp,checkpoint_dir_name,...,pid,hostname,node_ip,time_since_restore,iterations_since_restore,config/train_loop_config/depth,config/train_loop_config/ffn_hidden_dim,config/train_loop_config/ffn_num_layers,config/train_loop_config/message_hidden_dim,logdir
0,0.099042,0.168217,0.861368,0.700675,0.741955,0.099042,19,40,1729602279,checkpoint_000019,...,24873,Knathan-Laptop,172.31.231.162,49.881516,20,2,2000,2,500,f1a6e41a
1,0.069695,0.119898,0.902580,0.717637,0.814650,0.069695,19,40,1729602299,checkpoint_000019,...,24953,Knathan-Laptop,172.31.231.162,56.653336,20,2,2200,2,400,d775c15d


In [ ]:
# best configuration
best_result = results.get_best_result()
best_config = best_result.config
best_config['train_loop_config']

{'depth': 2,
 'ffn_hidden_dim': 2000,
 'ffn_num_layers': 2,
 'message_hidden_dim': 500}

In [ ]:
# best model checkpoint path
best_result = results.get_best_result()
best_checkpoint_path = Path(best_result.checkpoint.path) / "checkpoint.ckpt"
print(f"Best model checkpoint path: {best_checkpoint_path}")

Best model checkpoint path: /home/knathan/chemprop/examples/hpopt/ray_results/TorchTrainer_2024-10-22_09-03-37/f1a6e41a/checkpoint_000019/checkpoint.ckpt


In [ ]:
ray.shutdown()